In [93]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

df_train = pd.read_csv('../../data/for_modelling/credit_score_train.csv')
df_val = pd.read_csv('../../data/for_modelling/credit_score_valid.csv')
df=pd.concat([df_train, df_val])
target='DEFAULT'
#df=df_train.copy()
df.head()

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.00,0.0000,103,74,0.7184,0.0370,...,2.5003,0.0000,No,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.25,1.5882,35861,29157,0.8131,0.1141,...,0.5882,0.3704,High,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.20,5.3750,3716,2533,0.6816,0.0230,...,0.3125,0.0581,No,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.30,0.2131,637,187,0.2936,0.0398,...,0.1639,0.7692,No,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.40,1.8356,5276,2325,0.4407,0.0273,...,0.1370,0.0746,No,1,1,0,1,1,552,0


In [94]:
# columns to remove from EDA
columns_to_remove = ["T_CLOTHING_12","T_ENTERTAINMENT_12", "T_GROCERIES_12", "T_GROCERIES_6", "T_HEALTH_12", "T_TAX_12", "T_TAX_6", "T_TRAVEL_12", "T_TRAVEL_6","T_UTILITIES_12", "T_UTILITIES_6", "T_EXPENDITURE_12", "T_EXPENDITURE_6"]
columns_to_remove

['T_CLOTHING_12',
 'T_ENTERTAINMENT_12',
 'T_GROCERIES_12',
 'T_GROCERIES_6',
 'T_HEALTH_12',
 'T_TAX_12',
 'T_TAX_6',
 'T_TRAVEL_12',
 'T_TRAVEL_6',
 'T_UTILITIES_12',
 'T_UTILITIES_6',
 'T_EXPENDITURE_12',
 'T_EXPENDITURE_6']

In [95]:

# map CAT_GAMBLING (No-0, Low-1, High-2)
df_train['CAT_GAMBLING'] = df_train['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
df_val['CAT_GAMBLING'] = df_val['CAT_GAMBLING'].map({'No':0, 'Low':1, 'High':2})
# remove columns with high correlation
df_train.drop(columns=columns_to_remove, inplace=True)
df_val.drop(columns=columns_to_remove, inplace=True)
# remove outliers
from pyod.models.knn import KNN
clf = KNN(contamination=0.04)
rest_cols = [col for col in df_train.columns if col not in columns_to_remove]
# remove DEFAULT column
rest_cols.remove('DEFAULT')
clf.fit(df_train[rest_cols])
df_train['outliers'] = clf.labels_
# sum of outliers
df_train['outliers'].value_counts()

outliers
0    470
1     20
Name: count, dtype: int64

In [96]:
df_train = df_train[df_train['outliers'] == 0]
df_train.drop(columns='outliers', inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,R_CLOTHING_SAVINGS,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,2783,1855,0,0.6665,0.0000,0.0000,74,0.7184,0.0370,0.0555,...,2.5003,0.0000,0,0,0,0,1,0,570,0
1,314430,445442,707468,1.4167,2.2500,1.5882,29157,0.8131,0.1141,0.0805,...,0.5882,0.3704,2,1,0,1,1,0,691,0
2,161773,517674,2782496,3.2000,17.2000,5.3750,2533,0.6816,0.0230,0.0072,...,0.3125,0.0581,0,1,1,1,1,1,520,0
3,16014,97685,20818,6.1000,1.3000,0.2131,187,0.2936,0.0398,0.0065,...,0.1639,0.7692,0,1,0,0,1,0,654,0
4,193225,1410542,2589215,7.3000,13.4000,1.8356,2325,0.4407,0.0273,0.0037,...,0.1370,0.0746,0,1,1,0,1,1,552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,20007,22508,457665,1.1250,22.8752,20.3334,861,0.5934,0.0725,0.0645,...,1.1111,0.0546,0,1,0,0,1,0,423,0
466,20846,268679,23162,12.8888,1.1111,0.0862,533,0.4094,0.0625,0.0048,...,0.0862,1.0000,1,1,0,0,1,0,584,0
467,89235,1106514,160623,12.4000,1.8000,0.1452,835,0.3460,0.0270,0.0022,...,0.0806,0.5556,0,1,0,0,1,0,631,0
468,197073,1021195,1863233,5.1818,9.4545,1.8246,3948,0.4691,0.0427,0.0082,...,0.1754,0.0962,2,1,0,1,1,1,572,0


In [97]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from scipy.stats import boxcox

X=df_train.drop(target, axis=1)
X=X[rest_cols]
y=df_train[target]
X_val=df_val.drop(target, axis=1)
X_val=X_val[rest_cols]
y_val=df_val[target]
X=X[rest_cols]
X_cols=X[rest_cols]
# Box Cox and standart scalling
for col in rest_cols:
    X[col] = boxcox(X[col]+1)[0]
    X_val[col] = boxcox(X_val[col]+1)[0]
X[rest_cols] = scaler.fit_transform(X[rest_cols])
X_val[rest_cols] = scaler.transform(X_val[rest_cols])


In [98]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier

model1 = DecisionTreeClassifier(random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
model4 = SVC(random_state=42, kernel='poly', degree=10)

estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model3)]

In [9]:
from sklearn.ensemble import VotingClassifier
model_hard = VotingClassifier(estimators=estimators, voting='hard')
model_hard.fit(X,y)

y_hat = model_hard.predict(X_val)
print('accuracy: ', recall_score(y_val, y_hat))

print('model.score: ', model_hard.score(X_val,y_val))

accuracy:  0.5897435897435898
model.score:  0.427536231884058


In [10]:
model_soft = VotingClassifier(estimators=estimators, voting='soft')
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
recall_score(y_val, y_hat)

0.5897435897435898

In [11]:
from sklearn.metrics import classification_report
model_soft = VotingClassifier(estimators=estimators, voting='soft', weights=[0.05, 0.05, 0.90])
model_soft.fit(X, y)

y_hat = model_soft.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        99
           1       0.28      1.00      0.44        39

    accuracy                           0.28       138
   macro avg       0.14      0.50      0.22       138
weighted avg       0.08      0.28      0.12       138


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X, y).score(X_val, y_val)

0.2826086956521739

In [13]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(n_estimators=10, random_state=0)
clf.fit(X, y)
y_hat = clf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.29      0.43        99
           1       0.32      0.85      0.46        39

    accuracy                           0.45       138
   macro avg       0.57      0.57      0.45       138
weighted avg       0.68      0.45      0.44       138


In [14]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X, y)
y_hat = model_rf.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.72      1.00      0.84        99
           1       0.00      0.00      0.00        39

    accuracy                           0.72       138
   macro avg       0.36      0.50      0.42       138
weighted avg       0.51      0.72      0.60       138


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.31      0.43        99
           1       0.28      0.67      0.39        39

    accuracy                           0.41       138
   macro avg       0.49      0.49      0.41       138
weighted avg       0.58      0.41      0.42       138


In [16]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=1,
                                   learning_rate=0.01)
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.70      0.54      0.61        99
           1       0.26      0.41      0.32        39

    accuracy                           0.50       138
   macro avg       0.48      0.47      0.46       138
weighted avg       0.57      0.50      0.52       138


In [17]:
from xgboost import XGBClassifier
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X, y)
y_hat = model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        99
           1       0.38      0.38      0.38        39

    accuracy                           0.65       138
   macro avg       0.57      0.57      0.57       138
weighted avg       0.65      0.65      0.65       138


In [20]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
level0 = [
    ('lr', LogisticRegression()),
    ('svc', SVC()),
    ('dt', DecisionTreeClassifier())
]
# Meta-model, który podsumowuje przewidywania bazowych modeli
level1 = LogisticRegression()

# Defining the stacking model
model_stacking = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [19]:
# fitting stacking and evaluating
model_stacking.fit(X, y)
predictions = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, predictions))

Accuracy: 0.6521739130434783


## strojenie parametrów

In [21]:
tree_model= DecisionTreeClassifier()

In [26]:
from sklearn.model_selection import GridSearchCV

In [29]:
tree_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [34]:
max_depth=[3, 5, 6, 7, 8]
criterion=["gini","entrophy"]
ccp_alpha=[0,0.01, 0.02, 0.05, 0.1, 0.2]
param_grid = dict(max_depth=max_depth,criterion=criterion, ccp_alpha=ccp_alpha)

In [35]:
tree_model= DecisionTreeClassifier()
grid = GridSearchCV(estimator=tree_model, param_grid=param_grid, cv = 3, n_jobs=-1)

grid_result = grid.fit(X, y) #tutaj lepiej zastosować tylko trainset

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.744733 using {'ccp_alpha': 0.02, 'criterion': 'gini', 'max_depth': 3}


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Dell\anaconda3\Lib\site-packages\skle

In [49]:
std_test_scores = grid_result.cv_results_['std_test_score']
mean_test_scores = grid_result.cv_results_['mean_test_score']
parameters = grid_result.cv_results_['params']

for mean_test, std_test, params in zip(mean_test_scores, std_test_scores, parameters):
    print('Parameters : {}'.format(params))
    print('Mean test score: {:.8f}'.format(mean_test))
    print('Standard Deviation (test) : {:.8f}'.format(std_test))
    print()

print('Best parameters : {}'.format(grid_result.best_params_))
print('Best accuracy score : {:.8f}'.format(grid_result.best_score_))


Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3}
Mean test score: 0.70220208
Standard Deviation (test) : 0.02794190

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 5}
Mean test score: 0.68513528
Standard Deviation (test) : 0.01616429

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 6}
Mean test score: 0.69146388
Standard Deviation (test) : 0.01151412

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 7}
Mean test score: 0.67445152
Standard Deviation (test) : 0.00608799

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 8}
Mean test score: 0.66591812
Standard Deviation (test) : 0.03867906

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 3}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 5}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 6}
Mean

In [50]:
best_model=grid_result.best_estimator_
best_model.score(X_val, y_val)

0.2826086956521739

In [51]:
best_model.fit(X, y)
y_hat = best_model.predict(X_val)
report = classification_report(y_val, y_hat)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        99
           1       0.28      1.00      0.44        39

    accuracy                           0.28       138
   macro avg       0.14      0.50      0.22       138
weighted avg       0.08      0.28      0.12       138


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
from sklearn.model_selection import RandomizedSearchCV

In [52]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.717023 using {'ccp_alpha': 3.180672781924487, 'max_depth': 1}


In [53]:
best_model=random_result.best_estimator_
best_model.score(X_val, y_val)

0.717391304347826

In [55]:
from scipy.stats import poisson,expon
param_grid = {'ccp_alpha': expon(0.08),
              'max_depth': poisson(5)}

In [56]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1, random_state=123)

random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.717023 using {'ccp_alpha': 1.2722721434811057, 'max_depth': 7}


In [46]:
! pip install scikit-optimize

  Obtaining dependency information for scikit-optimize from https://files.pythonhosted.org/packages/90/0e/15deb91b3db0003843e34e72fa865e1d92013781d986fdc65483c99a9f69/scikit_optimize-0.10.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyaml>=16.9 from https://files.pythonhosted.org/packages/70/d9/6fdc01cee01fe56fddfd46892d9846ebb53a5f8eb78056955c516eef42c5/pyaml-24.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   ----------- --------------------------- 30.7/107.7 kB 660.6 kB/s eta 0:00:01
   -------------------------------------- - 102.4/107.7 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- 107.7/107.7 kB 894.5 kB/s eta 0:00:00


In [57]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

opt = BayesSearchCV(
    SVC(),
    {
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=5, #generalnie powinno być więcej, ale wtedy długo trwa
    cv=3
)

opt.fit(X, y)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_val, y_val))

val. score: 0.7255566443464533
test score: 0.2826086956521739


In [63]:
!pip install -U lightautoml

  Obtaining dependency information for lightautoml from https://files.pythonhosted.org/packages/6d/82/81771067b1ecc2ac87aec5d14d862b4567eea95504ef576718f6b7755ac6/lightautoml-0.3.8.1-py3-none-any.whl.metadata
  Using cached lightautoml-0.3.8.1-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for autowoe>=1.2 from https://files.pythonhosted.org/packages/61/09/ccf2ad21f105d07d73bedf065597359a2445d053696a9e1e57dcee397a97/AutoWoE-1.3.2-py3-none-any.whl.metadata
  Using cached AutoWoE-1.3.2-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for catboost>=0.26.1 from https://files.pythonhosted.org/packages/e8/37/3afd3c02798734efcd7840bfa872d3efc06f5d5c92f9613fea3ff5b4311f/catboost-1.2.3-cp311-cp311-win_amd64.whl.metadata
  Using cached catboost-1.2.3-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/4d/c1/9190d856fb2c9ca7e5dabab530289bf66d03c0e379096678c0ee210abf75/ho

In [64]:
import os
import time
import re

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Imports from LightAutoML package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

In [65]:
def f1_metric(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

def acc_score(y_true, y_pred):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int))

def gini(y_true, y_pred):
    return 2*roc_auc_score(y_true, y_pred)-1

task = Task('binary', metric = f1_metric)

In [72]:
roles = {'target': 'DEFAULT'}

In [73]:
automl = TabularAutoML(task = task,
                       timeout = 600, # 600 seconds = 10 minutes
                       cpu_limit = 4, # 
                       general_params = {'use_algos': [['linear_l2',
                                                        'lgb', 'lgb_tuned']]})

In [82]:
tr_data=X
tr_data["DEFAULT"]=y
valid_data=X_val
valid_data["DEFAULT"]=y_val
oof_pred = automl.fit_predict(tr_data, roles = roles)

In [83]:
valid_pred = automl.predict(valid_data)

In [85]:
print('OOF acc: {}'.format(acc_score(tr_data['DEFAULT'].values,      oof_pred.data[:, 0])))
print('VAL acc: {}'.format(acc_score(valid_data['DEFAULT'].values, valid_pred.data[:, 0])))

OOF acc: 0.7489361702127659
VAL acc: 0.644927536231884


In [86]:
!pip install tpot

In [87]:
from tpot import TPOTClassifier

In [88]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X, y)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Best pipeline: GaussianNB(input_matrix)


TPOTClassifier(generations=5, population_size=50, random_state=42, verbosity=2)

In [89]:
print("Accuracy on testing data: ", tpot.score(X_val, y_val))

Accuracy on testing data:  0.717391304347826


In [90]:
!pip install boruta

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

array([[-1.67311484, -1.79479368, -2.13887769, ...,  0.06537205,
        -0.42532872, -0.43113655],
       [ 1.52579493,  0.52768342,  0.49449643, ...,  0.06537205,
        -0.42532872,  1.9855153 ],
       [ 0.6954251 ,  0.66687435,  1.55375508, ...,  0.06537205,
         2.35112266, -1.1557775 ],
       ...,
       [ 0.10246894,  1.48375   , -0.32246311, ...,  0.06537205,
        -0.42532872,  0.66123881],
       [ 0.92176584,  1.38769659,  1.20700545, ...,  0.06537205,
         2.35112266, -0.39908084],
       [ 0.49070335, -0.6318861 ,  0.46881365, ...,  0.06537205,
        -0.42532872,  0.25773124]])